In [7]:
import random
# import pandas as pd
# from IPython.display import display, HTML

import torch
from torch import utils
from torch.utils import data
from torch.utils.data import Dataset, DataLoader

import multiprocessing

import accelerate
from accelerate import Accelerator

# import huggingface_hub
from transformers import T5ForConditionalGeneration, T5Config, T5TokenizerFast, T5Tokenizer, AutoTokenizer
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer, default_data_collator, get_scheduler
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


import datasets
from datasets import load_dataset #, load_from_disk
# import evaluate
# from evaluate import load


import tqdm as notebook_tqdm
import os
from dotenv import load_dotenv

from tqdm.auto import tqdm
import math

In [8]:
load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
# huggingface_token = os.getenv("HF_TOKEN")

device_type = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device_type)
# print(torch.cuda.device_count())
cpu_cores = multiprocessing.cpu_count()
# print(cpu_cores)
# device = torch.device("cpu")

accelerator = Accelerator()
device = accelerator.state.device

In [9]:
model_checkpoint = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [10]:
raw_dataset = load_dataset("xsum")

In [11]:
train_size = 1000
test_size = int(0.1 * train_size)
downsampled_dataset = raw_dataset["train"].train_test_split(train_size=train_size, test_size=test_size)

In [12]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = downsampled_dataset.map(preprocess_function, batched=True, num_proc=(cpu_cores-1))
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

Map (num_proc=7):   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=7):   0%|          | 0/100 [00:00<?, ? examples/s]

/root/miniforge3/envs/base_ggl/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
